In [1]:
import cPickle
import os
import numpy as np
import nltk
import nltk.corpus
import pandas as pd
import sys
import multiprocessing as mp
import text_tok
import random

from nltk import sent_tokenize
from ast import literal_eval
from datetime import datetime

# Set the number of cores to use
NUM_CORE = mp.cpu_count()-4

# Truecasing

A small fraction of the dpa articles were missing all uppercase letters. To fix this, we used a truecasing model by Nils Reimers
(https://github.com/nreimers/truecaser), which is based on the work of Lucian Vlad Lita et al. (2003).

In [2]:
# Set the path to the truecasing model
path = os.path.join(os.getcwd(), 'truecaser-master')
os.chdir(path)
sys.path.insert(1, os.getcwd().replace('\\truecaser-master', ''))

In [3]:
from TrainFunctions import *
from EvaluateTruecaser import evaluateTrueCaser

In [4]:
# Initialize frequency distributions for bigrams and trigrams
uniDist = nltk.FreqDist()
backwardBiDist = nltk.FreqDist() 
forwardBiDist = nltk.FreqDist() 
trigramDist = nltk.FreqDist() 
wordCasingLookup = {}

In [5]:
# Load training data
data = pd.read_csv(os.path.join(os.getcwd().replace('truecaser-master', ''), 'dpa_prepro_step11.csv'), encoding = 'utf-8', index_col = 0,  keep_default_na=False,
                   dtype = {'rubrics': 'str', 
                            'source': 'str',
                            'keywords': 'str',
                            'title': 'str',
                            'city': 'str',
                            'genre': 'str',
                            'wordcount': 'str'},
                  converters = {'paragraphs': literal_eval})

In [6]:
# Select the subset of data (500,000 articles) for training the truecasing model
data_sample = data['texts'][1500000:2000000]

During the preprocessing of the training set, texts are tokenized and umlauts are replaced with their respective non-umlaut equivalents using the function `text_tok`. This step is necessary as the dataset to which the model will be applied has umlauts already replaced.

In [7]:
startTime = datetime.now()

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    tokens = pool.map(text_tok.text_tok, [text for text in data_sample]) 
    pool.close()
    pool.join()
    
print(datetime.now()-startTime)

0:02:02.166000


In [8]:
startTime = datetime.now()

# Train the truecasing model using 500,000 articles from the dpa corpus, where the capitalization is correct
updateDistributionsFromSentences(tokens, wordCasingLookup, uniDist, backwardBiDist, forwardBiDist, trigramDist)

# Save the frequency distributions
f = open('distributions.obj', 'wb')
cPickle.dump(uniDist, f, -1)
cPickle.dump(backwardBiDist, f, -1)
cPickle.dump(forwardBiDist, f, -1)
cPickle.dump(trigramDist, f, -1)
cPickle.dump(wordCasingLookup, f, -1)
f.close()


print(datetime.now()-startTime)

0:42:29.351000


Evaluate the truecasing model using 10 manually chosen sentences.

In [9]:
testSentences = [u"Obwohl der Erwerb von Wohnungen seit 1974 f\xfcr Ausl\xe4nder weitgehend verboten ist, h\xe4tten diese in zahlreichen F\xe4llen \xfcber \xf6sterreichische Strohm\xe4nner Wohnungen gekauft."
                ,u"So seien \xabMenschen im \xf6sterreichischen Altersheim\xbb als K\xe4ufer aufgetreten, die ihr \xabEigentum\xbb dem deutschen Anleger \xabvererbt\xbb h\xe4tten."
                ,u"Die Zusammensetzung der k\xfcnftigen SPD-Fraktionsspitze um den neuen Vorsitzenden Hans-Ulrich Klose nimmt allm\xe4hlich Konturen an."
                ,u"Die bei der Wahl des Vorsitzenden unterlegenen Mitbewerber Herta D\xe4ubler-Gmelin und Rudolf Dre\xdfler sollen dem Vernehmen nach zum k\xfcnftigen Team um Klose geh\xf6ren."
                ,u"Dieser will seine Pl\xe4ne f\xfcr eine gestraffte Fraktionsf\xfchrung am 10. Dezember den sozialdemokratischen Abgeordneten zur Abstimmung vorlegen."
                ,u"Sie sollen nicht mehr wie bisher gleichzeitig als Arbeitskreisleiter f\xfcr bestimmte Fachgebiete zust\xe4ndig sein, sondern zu allen Themen \xabvom Paragraphen 218 bis zur Abschaffung der Gewerbekapitalsteuer\xbb kompetent Stellung nehmen k\xf6nnen."
                ,u"Es sei denkbar, da\xdf es auch Gegenkandidaten zu den Vorschl\xe4gen Kloses geben wird."
                ,u"D\xe4ubler-Gmelin und Dre\xdfler w\xfcrden \xabmit Sicherheit eine herausgehobene Rolle haben, wenn sie es wollen\xbb, sagte Struck."
                ,u"Parteichef Bj\xf6rn Engholm sagte vor dem Parteirat zu den Verlierern der Wahl vor zehn Tagen: \xabIch w\xfcnsche mir, da\xdf Ihr beide und Hans-Ulrich Klose ein enges Team bildet, um zu zeigen: Sieger und Unterlegene ziehen an einem Strang."
                ,u"Bei der Leitung der Arbeitskreise, deren Zahl ebenfalls deutlich verringert werden soll, k\xf6nnten \xabneue K\xf6pfe\xbb in die F\xfchrung kommen. Zur Disposition st\xfcnden grunds\xe4tzlich folgende Politiker und Arbeitsgebiete: D\xe4ubler-Gmelin (Recht), Dre\xdfler (Soziales), Matth\xe4us-Maier (Finanzen), Norbert Gansel (Ausw\xe4rtiges), Willfried Penner (Inneres) Ingrid Becker-Inglau (Frauen), Harald Sch\xe4fer (Umwelt), Wolfgang Roth (Wirtschaft) und Thierse (ohne Arbeitskreis)."
                ]

In [10]:
evaluateTrueCaser(testSentences, wordCasingLookup, uniDist, backwardBiDist, forwardBiDist, trigramDist)

[u'Obwohl', u'der', u'Erwerb', u'von', u'Wohnungen', u'seit', u'1974', u'f\xfcr', u'Ausl\xe4nder', u'weitgehend', u'verboten', u'ist', u',', u'h\xe4tten', u'diese', u'in', u'zahlreichen', u'F\xe4llen', u'\xfcber', u'\xf6sterreichische', u'Strohm\xe4nner', u'Wohnungen', u'gekauft', u'.']
[u'Obwohl', u'der', u'Erwerb', u'von', u'Wohnungen', u'seit', u'1974', u'F\xfcr', u'Ausl\xe4nder', u'weitgehend', u'verboten', u'ist', u',', u'H\xe4tten', u'diese', u'in', u'zahlreichen', u'F\xe4llen', u'\xdcber', u'\xd6sterreichische', u'Strohm\xe4nner', u'Wohnungen', u'gekauft', u'.']
-------------------
[u'So', u'seien', u'\xab', u'Menschen', u'im', u'\xf6sterreichischen', u'Altersheim', u'\xbb', u'als', u'K\xe4ufer', u'aufgetreten', u',', u'die', u'ihr', u'\xab', u'Eigentum', u'\xbb', u'dem', u'deutschen', u'Anleger', u'\xab', u'vererbt', u'\xbb', u'h\xe4tten', u'.']
[u'So', u'seien', u'\xab', u'Menschen', u'im', u'\xd6sterreichischen', u'Altersheim', u'\xbb', u'als', u'K\xe4ufer', u'aufgetreten', u

Evaluate the truecasing model using 100 randomly chosen sentences from 1000 articles that were not utilized in the model training process.

In [11]:
data_test = data['texts'][2000000:2001000]

data_test = data_test.tolist()
sentences = []

# Split the texts into sentences
for text in data_test:
    sentences.extend(sent_tokenize(text))
    
# Pick 100 random sentences
random_sentences = random.sample(sentences, 100)

In [12]:
evaluateTrueCaser(random_sentences, wordCasingLookup, uniDist, backwardBiDist, forwardBiDist, trigramDist)

[u'Trotzdem', u'unterst\xfctzen', u'rund', u'sieben', u'Prozent', u'der', u'befragten', u'Muslime', u'den', u'umstrittenen', u'Kandidaten', u'.']
[u'Trotzdem', u'Unterst\xfctzen', u'rund', u'sieben', u'Prozent', u'der', u'Befragten', u'Muslime', u'den', u'umstrittenen', u'Kandidaten', u'.']
-------------------
[u'Die', u'europ\xe4ische', u'und', u'deutsche', u'Gesetzgebung', u'verpflichtet', u'b\xf6rsennotierte', u'Unternehmen', u',', u'ihre', u'Anleger', u'umgehend', u'\xfcber', u'Entwicklungen', u'zu', u'informieren', u',', u'welche', u'die', u'Aktienkurse', u'wahrscheinlich', u'beeinflussen', u'werden', u'.']
[u'Die', u'Europ\xe4ische', u'und', u'deutsche', u'Gesetzgebung', u'verpflichtet', u'B\xf6rsennotierte', u'Unternehmen', u',', u'ihre', u'Anleger', u'umgehend', u'\xdcber', u'Entwicklungen', u'zu', u'informieren', u',', u'welche', u'die', u'Aktienkurse', u'wahrscheinlich', u'beeinflussen', u'werden', u'.']
-------------------
[u'\xab', u'Das', u'Krankenhaus', u'war', u'gepl\xfc

In [13]:
from Truecaser import *
import nltk
import string
import PredictTruecaser
from nltk.tokenize.treebank import TreebankWordDetokenizer
from sacremoses import MosesTokenizer, MosesDetokenizer # detokenizing package
md = MosesDetokenizer() 

C:\ProgramData\Anaconda3\envs\py27\lib\site-packages\sacremoses\truecase.py:26: UserWarning: You should really be using Python3!!! Tick tock, tick tock, https://pythonclock.org/
  "You should really be using Python3!!! "


In [14]:
# Define a test sentence for demonstration
test_sentence = 'dieses wort ist falsch geschrieben. dieser satz ist ein test.'

In [15]:
corr_sent = getTrueCase(nltk.word_tokenize(test_sentence), 'title', wordCasingLookup, uniDist, backwardBiDist, forwardBiDist, 
                       trigramDist)
corr_sent = md.detokenize(corr_sent).replace(u"``", u' " ').replace(u"''", u' " ').replace(u"'", u" ")
print(corr_sent)

Dieses Wort ist falsch geschrieben. Dieser Satz ist ein Test.


In [16]:
# Load articles that we want to correct
data_truecase = pd.read_csv(os.getcwd().replace('truecaser-master', '') + 'dpa_case_fix.csv', encoding = 'utf-8', sep=';', index_col = 0,  keep_default_na=False,
                   dtype = {'rubrics': 'str', 
                            'source': 'str',
                            'keywords': 'str',
                            'title': 'str',
                            'city': 'str',
                            'genre': 'str',
                            'wordcount': 'str'},
                  converters = {'paragraphs': literal_eval})

In [17]:
# An example of an article with incorrect capitalization
data_truecase['texts'].iloc[15]

u"wall street schliesst fester. new york (vwd) - die aktienboerse in new york konnte am donnerstag ihre im sitzungsverlauf gesehenen kursgewinne nahezu behaupten. der dow-jones-index fuer 30 industriewerte schloss mit 2.643,07 um 24,01 punkte bzw 0,92 prozent ueber vortagesschluss. auch die breiter gefassten marktbarometer sowie die branchenindizes lagen im plusterritorium. umgesetzt wurden 223,15 (169,44) millionen aktien, wobei die kursgewinner gegenueber den -verlierern eine fuehrung von rund drei zu eins behaupteten. haendler fuehrten die aufschlaege auf hoffnungen am markt zurueck, wonach das federal reserve board in kuerze weitere zinssenkungen vornehmen werde, um die us-konjunktur anzukurbeln. die hoffnungen waren am mittwoch durch aeusserungen von fed-chairman alan greenspan genaehrt worden, der angesichts des 'credit crunch' der banken in den usa eine senkung der us-tagesgeldsaetze befuerwortet hatte. ausserdem, so die boersianer, habe der markt eine stuetze durch die wieder n

In [18]:
# Use the trained truecasing model to correct articles
truecase_art = []

for text in data_truecase['texts']:
    truecase_art.append(getTrueCase(nltk.word_tokenize(text), 'title', wordCasingLookup, uniDist, backwardBiDist, forwardBiDist, trigramDist))

In [19]:
# Detokenize the articles
truecase_art = [md.detokenize(article).replace(u"``", u' " ').replace(u"''", u' " ').replace(u"'", u" ") for article in truecase_art]

In [20]:
truecase_art[15]

u'Wall Street schliesst fester. New York (Vwd) - Die Aktienboerse in New York konnte am Donnerstag ihre im Sitzungsverlauf gesehenen Kursgewinne nahezu behaupten. Der Dow-Jones-Index fuer 30 Industriewerte schloss mit 2.643,07 um 24,01 Punkte bzw 0,92 Prozent ueber Vortagesschluss. Auch die breiter gefassten Marktbarometer sowie die Branchenindizes lagen im Plusterritorium. umgesetzt wurden 223,15 (169,44) Millionen Aktien, wobei die Kursgewinner gegenueber den -verlierern eine Fuehrung von rund drei zu eins behaupteten. Haendler fuehrten die Aufschlaege auf Hoffnungen am Markt zurueck, wonach das Federal Reserve Board in Kuerze weitere Zinssenkungen vornehmen werde, um die US-Konjunktur anzukurbeln. Die Hoffnungen waren am Mittwoch durch AEusserungen von Fed-Chairman Alan Greenspan genaehrt worden, der angesichts des Credit Crunch  der Banken in den USA eine Senkung der Us-Tagesgeldsaetze befuerwortet hatte. Ausserdem, so die Boersianer, habe der Markt eine Stuetze durch die wieder na

In [21]:
data_truecase['texts'] = truecase_art

In [22]:
data_truecase['texts'].iloc[15]

u'Wall Street schliesst fester. New York (Vwd) - Die Aktienboerse in New York konnte am Donnerstag ihre im Sitzungsverlauf gesehenen Kursgewinne nahezu behaupten. Der Dow-Jones-Index fuer 30 Industriewerte schloss mit 2.643,07 um 24,01 Punkte bzw 0,92 Prozent ueber Vortagesschluss. Auch die breiter gefassten Marktbarometer sowie die Branchenindizes lagen im Plusterritorium. umgesetzt wurden 223,15 (169,44) Millionen Aktien, wobei die Kursgewinner gegenueber den -verlierern eine Fuehrung von rund drei zu eins behaupteten. Haendler fuehrten die Aufschlaege auf Hoffnungen am Markt zurueck, wonach das Federal Reserve Board in Kuerze weitere Zinssenkungen vornehmen werde, um die US-Konjunktur anzukurbeln. Die Hoffnungen waren am Mittwoch durch AEusserungen von Fed-Chairman Alan Greenspan genaehrt worden, der angesichts des Credit Crunch  der Banken in den USA eine Senkung der Us-Tagesgeldsaetze befuerwortet hatte. Ausserdem, so die Boersianer, habe der Markt eine Stuetze durch die wieder na

In [23]:
data_truecase.to_csv(os.path.join(os.getcwd().replace('truecaser-master', ''), 'dpa_cases_fixed.csv'), encoding='utf-8-sig', sep = ';')